<a href="https://colab.research.google.com/github/Amanda0975/MT/blob/main/Riqueza_2021_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [1]:
import requests
from io import BytesIO
from zipfile import ZipFile
import pandas as pd

# Enlace del archivo en Dropbox
url = 'https://www.dropbox.com/scl/fi/ecy7lnydd0zszwvvmp8o2/data.csv.zip?dl=0&e=1&rlkey=zqyvspuxoqyxvrkr6bkjwtjik'

# Ajustar el enlace para descargar el archivo directamente
url = url.replace("?dl=0", "?dl=1")

# Descarga el archivo ZIP desde el enlace de Dropbox
response = requests.get(url)
response.raise_for_status()  # Verifica si la descarga fue exitosa

# Lee el archivo ZIP en memoria
zip_file = ZipFile(BytesIO(response.content))

# Extrae y lee el archivo CSV del archivo ZIP
with zip_file.open(zip_file.namelist()[0]) as csv_file:
    df = pd.read_csv(csv_file)

<ipython-input-1-01568c2e2492>:21: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


In [2]:
#Creación columna "Region_id"
# Definir un diccionario de mapeo para los valores de la columna "Región" a "region_id"
region_mapping = {
    'XV REGION ARICA Y PARINACOTA': 15,
    'I REGION DE TARAPACA': 1,
    'II REGION DE ANTOFAGASTA': 2,
    'III REGION DE ATACAMA': 3,
    'IV REGION COQUIMBO': 4,
    'V REGION VALPARAISO': 5,
    'XIII REGION METROPOLITANA': 13,
    'VI REGION DEL LIBERTADOR GENERAL BERNARDO O\'HIGGINS': 6,
    'VII REGION DEL MAULE': 7,
    'XVI REGION DE ÑUBLE': 16,
    'VIII REGION DEL BIO BIO': 8,
    'IX REGION DE LA ARAUCANIA': 9,
    'XIV REGION DE LOS RIOS': 14,
    'X REGION LOS LAGOS': 10,
    'XI REGION AYSEN DEL GENERAL CARLOS IBAÑEZ DEL CAMPO': 11,
    'XII REGION DE MAGALLANES Y LA ANTARTICA CHILENA': 12
}

# Aplicar el mapeo a la columna "Región" para crear la nueva columna "region_id" en el DataFrame "df"
df['region_id'] = df['Región'].map(region_mapping)

In [3]:
#Creación columna "ventas" punto medio
ventas_pm = {
    1: 0,
    2: 100,
    3: 400,
    4: 1500,
    5: 3700,
    6: 7500,
    7: 17500,
    8: 37500,
    9: 75000,
    10: 150000,
    11: 400000,
    12: 800000,
    13: 1000000
}

df['aporte'] = df['Tramo según ventas'].map(ventas_pm)

In [4]:
#Datos año 2021 y 2022
df=df.loc[df['Año comercial'].isin([2021, 2022])]
df=df.loc[df['Tipo de contribuyente'].isin(['PERSONA JURIDICA COMERCIAL'])]

In [5]:
df['riqueza'] = df.groupby('Comuna')['aporte'].transform('sum')

In [ ]:
df.nunique()

Año comercial                            2
RUT                                 920199
DV                                      11
Razón social                        885600
Tramo según ventas                      13
Número de trabajadores dependie       2318
Fecha inicio de actividades vige     10164
Fecha término de giro                 1076
Fecha primera inscripción de ac       8935
Tipo término de giro                     2
Tipo de contribuyente                    1
Subtipo de contribuyente                19
Tramo capital propio positivo           10
Tramo capital propio negativo           10
Rubro económico                         22
Subrubro económico                     234
Actividad económica                    663
Región                                  17
Provincia                               57
Comuna                                 347
R_PRESUNTA                               1
OTROS_REGIMENES                          5
region_id                               16
aporte     

#Dataframe TOP 3 Rubros económicos

In [6]:
# Crear un DataFrame pivote para desagrupar los valores de 'Rubro económico'
pivot_df = pd.pivot_table(df, index=['Comuna', 'region_id'], columns='Rubro económico', values='aporte', aggfunc='sum').reset_index()

# Llenar los valores NaN con 0
pivot_df.fillna(0, inplace=True)

# Calcular la suma de 'aporte' por 'Comuna' y 'region_id' para obtener la riqueza de cada 'Rubro económico'
for col in pivot_df.columns[2:]:
    pivot_df[col] = pivot_df[col].astype(int)

# Renombrar las columnas para que no tengan espacios en blanco
pivot_df.columns = pivot_df.columns.str.replace(' ', '_')

# Crear el DataFrame df1 con las columnas deseadas
df1 = pivot_df[['Comuna', 'region_id'] + list(pivot_df.columns[2:])]

# Mostrar el nuevo DataFrame df1
display(df1)


Rubro económico,Comuna,region_id,"ACTIVIDADES_ARTISTICAS,_DE_ENTRETENIMIENTO_Y_RECREATIVAS",ACTIVIDADES_DE_ALOJAMIENTO_Y_DE_SERVICIO_DE_COMIDAS,ACTIVIDADES_DE_ATENCION_DE_LA_SALUD_HUMANA_Y_DE_ASISTENCIA_SOCIAL,ACTIVIDADES_DE_LOS_HOGARES_COMO_EMPLEADORES;_ACTIVIDADES_NO_DIFERENCIADAS_DE_LOS_HOGARES,ACTIVIDADES_DE_ORGANIZACIONES_Y_ORGANOS_EXTRATERRITORIALES,ACTIVIDADES_DE_SERVICIOS_ADMINISTRATIVOS_Y_DE_APOYO,ACTIVIDADES_FINANCIERAS_Y_DE_SEGUROS,ACTIVIDADES_INMOBILIARIAS,...,CONSTRUCCION,ENSEÑANZA,EXPLOTACION_DE_MINAS_Y_CANTERAS,INDUSTRIA_MANUFACTURERA,INFORMACION_Y_COMUNICACIONES,OTRAS_ACTIVIDADES_DE_SERVICIOS,"SUMINISTRO_DE_AGUA;_EVACUACION_DE_AGUAS_RESIDUALES,_GESTION_DE_DESECHOS_Y_DESCONTAMINACION","SUMINISTRO_DE_ELECTRICIDAD,_GAS,_VAPOR_Y_AIRE_ACONDICIONADO",TRANSPORTE_Y_ALMACENAMIENTO,Valor_por_Defecto
0,ALGARROBO,5.0,5300,892900,113100,0,0,361100,5920800,3963900,...,636200,18700,400,384800,31200,16300,24300,1675100,84400,0
1,ALHUE,13.0,0,91400,11200,0,0,225200,114100,17500,...,13600,0,2150000,242600,1500,7400,0,1700,118000,0
2,ALTO BIO BIO,8.0,0,16000,1900,0,0,700,26500,67700,...,5600,0,0,3700,1900,0,0,25000,23300,0
3,ALTO DEL CARMEN,3.0,200,0,0,0,0,3100,0,11300,...,16200,100,0,99000,400,0,0,0,15200,0
4,ALTO HOSPICIO,1.0,28700,258700,168600,0,0,2956400,128000,895200,...,7722600,28100,1853700,4411700,281100,119000,269500,14900,7752500,4000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,VITACURA,13.0,4202600,23408500,27449400,0,0,28263600,179798400,117861400,...,61018800,5330400,11544500,35410800,20331300,5570800,1536600,8839800,15705500,90000
342,YERBAS BUENAS,7.0,100,118600,1900,0,0,84900,1900,21100,...,217400,200,0,1222300,11200,3700,90000,0,109100,0
343,YUMBEL,8.0,200,66400,9100,0,0,114100,157600,46100,...,172400,85200,7400,497700,8000,2000,3700,0,146500,0
344,YUNGAY,16.0,900,51100,15200,0,0,32200,37000,37700,...,354900,0,75000,368700,20000,3900,1800,0,172300,0


In [7]:
# Crear un DataFrame pivote para desagrupar los valores de 'Rubro económico'
pivot_df = pd.pivot_table(df, index=['Comuna', 'region_id'], columns='Rubro económico', values='aporte', aggfunc='sum').reset_index()

# Llenar los valores NaN con 0
pivot_df.fillna(0, inplace=True)

# Obtener los tres rubros económicos con los valores más altos
top3 = pivot_df.iloc[:, 2:].apply(lambda row: sorted(row.items(), key=lambda x: x[1], reverse=True)[:3], axis=1)

# Crear las columnas "TOP3" y "TOP3_valor"
pivot_df['TOP3'] = top3.apply(lambda x: ', '.join([item[0] for item in x]))

# Renombrar las columnas para que no tengan espacios en blanco
pivot_df.columns = pivot_df.columns.str.replace(' ', '_')

# Crear el DataFrame df1 con las columnas deseadas
df1 = pivot_df[['Comuna', 'region_id', 'TOP3']]

# Mostrar el nuevo DataFrame df1
display(df1)


Rubro económico,Comuna,region_id,TOP3
0,ALGARROBO,5.0,"ACTIVIDADES FINANCIERAS Y DE SEGUROS, ACTIVIDA..."
1,ALHUE,13.0,"EXPLOTACION DE MINAS Y CANTERAS, AGRICULTURA, ..."
2,ALTO BIO BIO,8.0,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...
3,ALTO DEL CARMEN,3.0,"AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA, ..."
4,ALTO HOSPICIO,1.0,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...
...,...,...,...
341,VITACURA,13.0,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...
342,YERBAS BUENAS,7.0,"INDUSTRIA MANUFACTURERA, COMERCIO AL POR MAYOR..."
343,YUMBEL,8.0,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...
344,YUNGAY,16.0,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...


#Merge

In [10]:
data=pd.merge(df1, df, on='Comuna', how='right')

In [21]:
#Filtrar columans relevantes
data=data.loc[:, ['Comuna', 'region_id_x', 'TOP3', 'Año comercial', 'riqueza']].drop_duplicates().dropna()

In [23]:
#Separar dataframe por año
data_2021 = data.loc[data['Año comercial'] == 2021, ['Comuna', 'region_id_x', 'TOP3', 'Año comercial', 'riqueza']]
data_2022 = data.loc[data['Año comercial'] == 2022, ['Comuna', 'region_id_x', 'TOP3', 'Año comercial', 'riqueza']]

#Data Description

In [26]:
#Top 5 comunas con mayor riqueza de 2022
data_2022.nlargest(5, 'riqueza')[['Comuna', 'TOP3', 'riqueza']]

,Comuna,TOP3,riqueza
790307,LAS CONDES,"ACTIVIDADES FINANCIERAS Y DE SEGUROS, COMERCIO...",4250579800
790257,PROVIDENCIA,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,1786016500
790263,SANTIAGO,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,1423054800
790322,VITACURA,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,848073400
790519,QUILICURA,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,602955000


In [25]:
#Top 5 comunas con menor riqueza de 2022
data_2022.nsmallest(5, 'riqueza')[['Comuna', 'TOP3', 'riqueza']]

,Comuna,TOP3,riqueza
1125503,GENERAL LAGOS,"INFORMACION Y COMUNICACIONES, AGRICULTURA, GAN...",300
961821,ANTARTIDA,"CONSTRUCCION, INDUSTRIA MANUFACTURERA, ACTIVID...",3100
1036415,TORTEL,"AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA, ...",10200
1055882,CAMINA,"CONSTRUCCION, TRANSPORTE Y ALMACENAMIENTO, AGR...",19900
817047,RIO VERDE,"AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA, ...",30300


In [27]:
#Top 5 comunas con mayor riqueza de 2021
data_2021.nlargest(5, 'riqueza')[['Comuna', 'TOP3', 'riqueza']]

,Comuna,TOP3,riqueza
51,LAS CONDES,"ACTIVIDADES FINANCIERAS Y DE SEGUROS, COMERCIO...",4250579800
0,PROVIDENCIA,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,1786016500
6,SANTIAGO,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,1423054800
66,VITACURA,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,848073400
271,QUILICURA,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,602955000


In [28]:
#Top 5 comunas con menor riqueza de 2022
data_2021.nsmallest(5, 'riqueza')[['Comuna', 'TOP3', 'riqueza']]

,Comuna,TOP3,riqueza
353665,GENERAL LAGOS,"INFORMACION Y COMUNICACIONES, AGRICULTURA, GAN...",300
179776,ANTARTIDA,"CONSTRUCCION, INDUSTRIA MANUFACTURERA, ACTIVID...",3100
258650,TORTEL,"AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA, ...",10200
279424,CAMINA,"CONSTRUCCION, TRANSPORTE Y ALMACENAMIENTO, AGR...",19900
27872,RIO VERDE,"AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA, ...",30300


In [29]:
# Agrupar los datos por "Comuna" y "Año comercial", y calcular las estadísticas de "riqueza"
stats_riqueza = data.groupby(['Comuna', 'Año comercial'])['riqueza'].agg(['nunique'])

# Filtrar para obtener las comunas donde hay más de un valor único de "riqueza"
comunas_con_diferencias = stats_riqueza[stats_riqueza['nunique'] > 1]

# Mostrar las comunas donde la "riqueza" es diferente entre "Año comercial"
print(comunas_con_diferencias)


Empty DataFrame
Columns: [nunique]
Index: []


In [22]:
display(data)

,Comuna,region_id_x,TOP3,Año comercial,riqueza
0,PROVIDENCIA,13.0,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,2021,1786016500
1,LA CISTERNA,13.0,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,2021,61563000
2,SAN BERNARDO,13.0,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,2021,320562400
3,LA SERENA,4.0,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,2021,152188800
5,PENCAHUE,7.0,"AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA, ...",2021,6128100
...,...,...,...,...,...
961821,ANTARTIDA,12.0,"CONSTRUCCION, INDUSTRIA MANUFACTURERA, ACTIVID...",2022,3100
1036415,TORTEL,11.0,"AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA, ...",2022,10200
1036519,OLLAGUE,2.0,"ACTIVIDADES ARTISTICAS, DE ENTRETENIMIENTO Y R...",2022,39700
1055882,CAMINA,1.0,"CONSTRUCCION, TRANSPORTE Y ALMACENAMIENTO, AGR...",2022,19900


#Exportar

In [30]:
# Guardar cada DataFrame en un archivo CSV
data_2022.to_csv('Riqueza_2021_2022.csv', index=False)